In [1]:
import os
import glob
import pickle
import numpy as np
logs_directory = 'logs'

# Get a list of all subdirectories in the logs directory
experiment_directories = [directory for directory in glob.glob(os.path.join(logs_directory, '*')) if os.path.isdir(directory)]

# Define a dictionary to store the grouped experiments
grouped_experiments = {}

# Iterate over each experiment directory
for experiment_directory in experiment_directories:
    # Parse the experiment name
    experiment_name = os.path.basename(experiment_directory)
    name_parts = experiment_name.split('_')
    
    # Extract the relevant values from the experiment name
    if 'naive' in name_parts:
        b = 0 if 'naive' in name_parts else -1#int(name_parts[-5])
    elif 'greedy' in name_parts and 'cvx' not in name_parts and int(name_parts[-4]) >=41:
        b = -1
    else:
        continue
    N = int(name_parts[-3])
    
    # Get the last pickle file in the 'data' subdirectory
    data_directory = os.path.join(experiment_directory, 'data')
    pickle_files = glob.glob(os.path.join(data_directory, '*.pkl'))
    last_pickle_file = max(pickle_files, key=os.path.getctime)
    
    # Load the last pickle file
    with open(last_pickle_file, 'rb') as file:
        data = pickle.load(file)
    
    # Find the 'summar' subdirectory and get the path to the text file
    summar_directory = os.path.join(experiment_directory, 'summary')
    text_files = glob.glob(os.path.join(summar_directory, '*.txt'))
    last_text_file = max(text_files, key=os.path.getctime)
    
    # Read the last line of the text file to extract coverage
    with open(last_text_file, 'r') as file:
        lines = file.readlines()
        last_line = lines[-2].strip()
        coverage = float(last_line.split()[1])

    # Group experiments by 'b' value
    if b not in grouped_experiments:
        grouped_experiments[b] = {}
    
    # Group experiments by 'N' value
    if N not in grouped_experiments[b]:
        grouped_experiments[b][N] = []
    
    # Add the experiment data to the grouped experiments dictionary
    grouped_experiments[b][N].append([
            np.sum(len(g) for g in data['rb']), coverage , data['ttotal']])

# Print the grouped experiments
for b, experiments in grouped_experiments.items():
    print(f"Experiments with b = {b}:")
    for N, experiment_data in experiments.items():
        print(f"  Experiments with N = {N}:")
        for datas in experiment_data:
            print(f"    Experiment data: {datas}")

Experiments with b = -1:
  Experiments with N = 500:
    Experiment data: [92, 0.8008, 1385.2676928043365]
    Experiment data: [101, 0.7886, 1852.1650879383087]
    Experiment data: [89, 0.7918, 1408.7474071979523]
    Experiment data: [91, 0.791, 1433.421273946762]
    Experiment data: [93, 0.7992, 1686.6987035274506]
    Experiment data: [83, 0.798, 1402.390331029892]
    Experiment data: [87, 0.7864, 1300.8102056980133]
    Experiment data: [91, 0.8116, 1536.713322877884]
    Experiment data: [104, 0.793, 2168.0915188789368]
    Experiment data: [108, 0.7922, 2332.5381438732147]
Experiments with b = 0:
  Experiments with N = 1:
    Experiment data: [211, 0.785, 2879.9784038066864]
    Experiment data: [210, 0.8027, 2932.3846418857574]
    Experiment data: [221, 0.781, 3046.202001094818]
    Experiment data: [185, 0.7883, 2283.041822910309]
    Experiment data: [183, 0.8047, 2356.5532450675964]
    Experiment data: [201, 0.7973, 2890.8788261413574]
    Experiment data: [196, 0.7793,

<ipython-input-1-e88be01954b5>:58: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  np.sum(len(g) for g in data['rb']), coverage , data['ttotal']])


In [2]:
data = np.array(grouped_experiments[-1][500])
mean = data.mean(axis=0)
std = data.std(axis=0)
print(f"regs {mean[0]:.1f} t {mean[2]:.1f}")
print(f"regs {std[0]:.1f} t {std[2]:.1f}")

regs 93.9 t 1650.7
regs 7.5 t 338.8
